# Kjøre Sas fra Python
Med Sas sit bibliotek saspy kan vi kjøre Sas fra Python. Vi kan også konvertere fra Sas-datasett til Pandas dataframe og omvendt

In [ ]:
import saspy
import os

Sjekker hvilken mappe vi jobber i 

In [ ]:
os.getcwd()

In [ ]:
os.environ['FELLES']

Sjekker hvor konfigurasjonsfilen til saspy ligger

In [ ]:
#sys.executable
saspy.SAScfg

## Starte en Sas-sesjon fra Python
Felles konfigurasjonsfil brukes. Se mer info her: https://wiki.ssb.no/pages/viewpage.action?pageId=187499307 (utenfor prod-sonen).

In [ ]:
felles = os.environ['FELLES']
#felles = '/ssb/bruker/felles'
# Gyldige verdier for cfgtype er ssh (denne spør ikke om passord, men virker ikke uten en engangssak) og iomlinux (denne spør om bruker og passord) 
# Ved ssh kan host-parameter brukes. Ved iomlinux kan iomhost-paremeter brukes. Hvis iomhost ikke brukes tildeles en tilfeldig server(!)
cfgtype= 'iomlinux'
SasServer = 'sl-sas-work-3'
#sas = saspy.SASsession(cfgname=cfgtype,cfgfile=f'{felles}/sascfg.py', encoding='latin1', host=SasServer)
sas = saspy.SASsession(cfgname=cfgtype,cfgfile=f'{felles}/sascfg.py', encoding='latin1')

### Sjekke hvilken server vi er på

In [ ]:
result = sas.submit(
"""
%put &=SYSTCPIPHOSTNAME;
""")
print(result['LOG'])

## Bruke Python-kode på Sas-datasett
Her definerer vi først datasettet som et eget objekt i Python. Deretter kan vi bruke Python-kode på det, her illustrert med head()

In [ ]:
demographics = sas.sasdata('demographics',libref='sashelp')
demographics.head()

For å jobbe med Sas-datasett som ikke er kjent for Sas må vi definere libname til mappen der Sas-datasettene ligger. Vi gjør det med sas.saslib. Her ser vi at vi kan bruke miljøvariable direkte. Dette er fordi det er startet en Sas-sesjon på Sas-serveren som er valgt og i Sas kan vi bruke miljøvariablene som $FELLES etc. Det kan vi ikke gjøre i vanlig Python.

Lager en libref til mappen med Sas-datasett. Libref blir datalib som angitt i parentes. Navnet libref er bare dere objektet som inneholder libref'en blir lagret.

In [ ]:
libref = sas.saslib('datalib',path='$FELLES/saskurs/data')

Utdraget fra Sas blir her et Sas-objekt og ikke en Pandas dataframe. Se neste eksempel for å konvertere til en Pandas dataframe.

In [ ]:
industri_2011 = sas.sasdata('industri_2011','datalib')
industri_2011.head()

## Konvertere fra Sas-datasett til Pandas dataframe
Vi bruker sas.sasdata2dataframe for å gjøre om et Sas-datasett til en Python dataframe

In [ ]:
industri_2011 = sas.sasdata2dataframe('industri_2011',libref='datalib')
industri_2011

### Med seleksjoner
Kan velge variabler med enten **keep** eller **drop**. Disse angis som Python dictionaries.

In [ ]:
opsjoner = {'keep': 'naring reg_type omsetning'}
industri_2011x = sas.sasdata2dataframe('industri_2011',libref='datalib', dsopts=opsjoner)
industri_2011x

Kan bruke **where** for å velge ut rader som tilfredstiller betingelser

In [ ]:
opsjoner = {'where': 'reg_type = "02"'}
industri_2011_r02 = sas.sasdata2dataframe('industri_2011',libref='datalib', dsopts=opsjoner)
industri_2011_r02

Kan kombinere flere opsjoner

In [ ]:
opsjoner = {'keep': 'naring reg_type omsetning', 'where': 'reg_type = "02"', 'obs': '111'}
industri_2011x_r02 = sas.sasdata2dataframe('industri_2011',libref='datalib', dsopts=opsjoner)
industri_2011x_r02

## Konvertere fra en Pandas dataframe til et Sas-datasett
For å gå den andre veien, fra Pandas dataframe til Sas-datasett bruker vi sas.dataframe2sasdata

In [ ]:
mitt_lib = sas.saslib('mitt_lib',path='$HOME/python/data')
sas.dataframe2sasdata(df=industri_2011,table='industri_2011_fra_python',libref='mitt_lib')

## Kjøre et innskutt Sas-program fra Python
Programmet skrives mellom trippelfnutter som vist under

In [ ]:
result = sas.submitLST(
"""
proc print data=mitt_lib.industri_2011_fra_python (obs=11) n;
 title "11 første obs fra sasdatasett laget fra python. Kjørt på &SYSTCPIPHOSTNAME";
run;   
""")

## Kjøre et eksternt Sas-program fra Python
Vi kan gjøre det med en %include-setning i det programmet vi sender til Sas

In [ ]:
result = sas.submitLST("%include '$HOME/python/prog/print.sas';")

## Sende en parameter til en makrovariabel i Sas
Når vi trenger å sende en eller flere parametere til Sas-programmet vi skal kjøre kan gjøre det ved hjelp av en sas.symput-kommando

In [ ]:
sas.symput('naring','10.110')
result = sas.submitLST(
"""
proc print data=mitt_lib.industri_2011_fra_python n;
 where naring = "&naring";
 title "Næring &naring fra sasdatasett laget fra python";
run;
""")

## Angi en parameter når programmet kjøres
Vi kan be om å få et vindu for å fylle inn en parameter til programmet ved å legge til prompt i kallet til Sas-programmet

In [ ]:
result = sas.submitLST(
"%include '$HOME/python/prog/print_m_makrovar.sas';",
prompt = {'naring': False})

Dette kan også gjøres med et innskutt program

In [ ]:
result = sas.submitLST(
"""
proc print data=mitt_lib.industri_2011_fra_python n;
 where naring = "&naring";
 title "Næring &naring fra sasdatasett laget fra python";
run;
""",
prompt = {'naring': False})

In [ ]:
result = sas.submitLST(
"%include '$HOME/python/prog/print_m_makrovar.sas';",
prompt = {'naring': False})

## Hente logg og list
Vi kan få logger og utlistinger sendt tilbake fra Sas for utlisting i Python. Vi kaller først opp display og HTML fra IPython

In [ ]:
from IPython.core.display import display, HTML

For å skrive ut loggen og list kan vi gjøre som vist under

In [ ]:
result = sas.submit(
"""
proc print data=mitt_lib.industri_2011_fra_python n;
 where naring = "&naring";
 title "Næring &naring fra sasdatasett laget fra python";
run;
""",
prompt = {'naring': False},method='listandlog')
print(result['LOG'])
display(HTML(result['LST']))